Greggs Vs Pret

---
---
---

In [14]:
import os
import configparser

import geopandas as gpd
from shapely.geometry import Point, Polygon

import requests
import json
import pandas as pd

import matplotlib.pyplot as plt

#for legends
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import math
from matplotlib.lines import Line2D

import folium

use of overpass adopted from https://github.com/Lisa-Ho/30-day-map-challenge/blob/main/2023/Day3-london-pubs/london-pubs.ipynb

In [7]:
# get Greggs from OSM using Overpass API
overpass_url = 'http://overpass-api.de/api/interpreter'
overpass_query = '''
[out:json];
area["ISO3166-1"="GB"]->.searchArea;

(way["brand:wikidata"="Q3403981"](area.searchArea);  
node["brand:wikidata"="Q3403981"](area.searchArea);  
rel["brand:wikidata"="Q3403981"](area.searchArea); 
);
out center;

'''
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [8]:
# convert json into dataframe
df_dict = {'id':[], 'type': [], 'lat':[], 'lon': [],'name':[]}

for element in data['elements']:
    df_dict['id'].append(element['id'])
    df_dict['type'].append(element['type'])
    
    for tag in list(df_dict.keys())[4:]:
        try:
            df_dict[tag].append(element['tags'][tag])
        except KeyError:
            df_dict[tag].append('') 
    if element['type']=='node':
        df_dict['lat'].append(element['lat'])
        df_dict['lon'].append(element['lon'])
    else:
        df_dict['lat'].append(element['center']['lat'])
        df_dict['lon'].append(element['center']['lon'])
        
df = pd.DataFrame.from_dict(df_dict)

# convert into geodataframe
geometry = [Point(xy) for xy in zip(df['lon'].astype(float), df['lat'].astype(float))]
greggs_gdf = gpd.GeoDataFrame(df, crs = 'EPSG:4326', geometry = geometry)

In [9]:
# get Prets from OSM using Overpass API
overpass_url = 'http://overpass-api.de/api/interpreter'
overpass_query = '''
[out:json];
area["ISO3166-1"="GB"]->.searchArea;

(way["brand:wikidata"="Q2109109"](area.searchArea);  
node["brand:wikidata"="Q2109109"](area.searchArea);  
rel["brand:wikidata"="Q2109109"](area.searchArea); 
);
out center;

'''
response = requests.get(overpass_url, params={'data': overpass_query})
pret_data = response.json()

In [10]:
# convert json into dataframe
df_dict = {'id':[], 'type': [], 'lat':[], 'lon': [],'name':[]}

for element in pret_data['elements']:
    df_dict['id'].append(element['id'])
    df_dict['type'].append(element['type'])
    
    for tag in list(df_dict.keys())[4:]:
        try:
            df_dict[tag].append(element['tags'][tag])
        except KeyError:
            df_dict[tag].append('') 
    if element['type']=='node':
        df_dict['lat'].append(element['lat'])
        df_dict['lon'].append(element['lon'])
    else:
        df_dict['lat'].append(element['center']['lat'])
        df_dict['lon'].append(element['center']['lon'])
        
df = pd.DataFrame.from_dict(df_dict)

# convert into geodataframe
geometry = [Point(xy) for xy in zip(df['lon'].astype(float), df['lat'].astype(float))]
pret_gdf = gpd.GeoDataFrame(df, crs = 'EPSG:4326', geometry = geometry)

In [11]:
#append the greggs and pret dataframe
appended_df = greggs_gdf.append(pret_gdf, ignore_index=True)


C:\Users\EMoses\AppData\Local\Temp\ipykernel_19340\3652567341.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_df = greggs_gdf.append(pret_gdf, ignore_index=True)


In [12]:
appended_df

,id,type,lat,lon,name,geometry
0,140697749,node,51.498230,-0.135216,Greggs,POINT (-0.13522 51.49823)
1,251977031,node,51.705522,-0.612239,Greggs,POINT (-0.61224 51.70552)
2,290369386,node,51.236173,-0.575119,Greggs,POINT (-0.57512 51.23617)
3,297118481,node,53.795863,-1.663838,Greggs,POINT (-1.66384 53.79586)
4,313192446,node,52.480509,-1.897402,Greggs,POINT (-1.89740 52.48051)
...,...,...,...,...,...,...
2034,997224975,way,51.494448,-0.212520,Pret A Manger,POINT (-0.21252 51.49445)
2035,1002485835,way,51.493027,-0.225577,Pret A Manger,POINT (-0.22558 51.49303)
2036,1011408798,way,51.462382,-0.216090,Pret A Manger,POINT (-0.21609 51.46238)
2037,1034909999,way,52.708407,-2.754437,Pret A Manger,POINT (-2.75444 52.70841)


In [15]:
# creating a folium map 

m = folium.Map(
    location=[52.595616, -1.336806], # start location as lat and lon
    tiles="CartoDB dark_matter", # adding a dark basemap
    zoom_start=7, # level of zoom
    prefer_canvas=True, # useful for changing the base map
)

# Create a dictionary to map names to colors
color_dict = {
    "Pret A Manger": '#8C240E',
    "Greggs": '#3186cc',
}

# Iterate through every row of the GeoDataFrame using the iterrows() function
for index, val in appended_df.iterrows():
    # Set color based on the value in the 'name' column, default to '#3186cc'
    fill_color = color_dict.get(val["name"])

    folium.CircleMarker(
        location=[val["geometry"].y, val["geometry"].x],  # Extracting lat and lon from Point geometry
        radius=5,
        popup=val["name"],
        color=fill_color,  # Outline color
        fill=True,
        fill_color=fill_color  # Fill color based on the 'name' column
    ).add_to(m)


m


In [16]:

# Save the map as an HTML file
m.save('index.html')